## Nilakshi Pokharkar

### Import Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import math
import warnings
warnings.filterwarnings("ignore")

### Load training data i.e train.dat

In [2]:
df_train = pd.read_csv('data/train.dat', delimiter=' ', names=['Year', 'Working_Age_Population'])
train_year = df_train['Year'].to_numpy().reshape(-1,1)
train_population = df_train['Working_Age_Population'].to_numpy().reshape(-1,1)

### Load testing data i.e test.dat

In [3]:
df_test = pd.read_csv('data/test.dat', delimiter=' ', names=['Year', 'Working_Age_Population'])
test_year = df_test['Year'].to_numpy().reshape(-1,1)
test_population = df_test['Working_Age_Population'].to_numpy().reshape(-1,1)

### Load train, test data and split into features and target variable and also initialize KFOLD

In [4]:
X = train_year
y = train_population

X_test = test_year
y_test = test_population

kf = KFold(n_splits = 6)

### Define the arrays for degrees from 0 to 12(inclusive) and alpha values

In [ ]:
degrees = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 

e = math.e 
alphas = [0, e ** -25, e ** -20, e ** -14, e ** -7, e ** -3, 1, e ** 3, e **7]

### Instantiate StandardScaler object 

In [ ]:
scaler = StandardScaler()

### To find the avg_rsme on each fold without lambda

In [ ]:
rmse_list = []
avg_rmse_list = []

rmse_list_train = []
avg_rmse_list_train = []
for degree in degrees:
    rmse_scores = []
    avg_rmse = 0
    
    rmse_scores_train = []
    avg_rmse_train = 0
  
    for train_index, val_index in kf.split(X):
        # Split the data into training and Validation sets
        X_train, X_Val = X[train_index], X[val_index]
        y_train, y_Val = y[train_index], y[val_index]
        
        # Using StandardScaler to Normalize data
        X_train_scaled = scaler.fit_transform(X_train)
        X_Val_scaled = scaler.transform(X_Val)
        
        y_train_scaled = scaler.transform(y_train)
        y_Val_scaled = scaler.transform(y_Val)
        
        # polynomial features
        poly = PolynomialFeatures(degree=degree)
        X_train_poly = poly.fit_transform(X_train_scaled)
        X_Val_poly = poly.transform(X_Val_scaled)
        
        # linear regression model
        model = LinearRegression()
        model.fit(X_train_poly, y_train_scaled)

        # predictions on the Validation test set
        y_pred_Val = model.predict(X_Val_poly)
        y_pred_Val_inv = scaler.inverse_transform(y_pred_Val)
        
        # predictions on the Train set
        y_pred_train = model.predict(X_train_poly)
        y_pred_train_inv = scaler.inverse_transform(y_pred_train)

        # To find the RMSE score 
        rmse = np.sqrt(mean_squared_error(y_Val, y_pred_Val_inv))
        rmse_scores.append(rmse)
        
        rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train_inv))
        rmse_scores_train.append(rmse_train)
        
        # Append the RMSE score to the average
        avg_rmse += rmse
        avg_rmse_train += rmse_train

    #these are the errorson the Val set of the training data
    avg_rmse /= kf.get_n_splits()
    rmse_list.append(rmse_scores)
    avg_rmse_list.append(avg_rmse)
    
    avg_rmse_train /= kf.get_n_splits()
    rmse_list_train.append(rmse_scores_train)
    avg_rmse_list_train.append(avg_rmse_train)

In [ ]:
#rmse_list

In [ ]:
avg_rmse_list

In [ ]:
# print the index of degree with the minimum avg_rsme.
min(range(len(avg_rmse_list)), key=lambda x : avg_rmse_list[x])

In [ ]:
plt.figure(figsize=(10, 6))   
plt.title("Graph 1: Avg RMSE vs.Degrees", size=16)
plt.scatter(degrees, avg_rmse_list)
plt.plot(degrees, avg_rmse_list, c="red", label='CV_test')
plt.scatter(degrees, avg_rmse_list_train)
plt.plot(degrees, avg_rmse_list_train, c="green", label='CV_train')
plt.ylabel("Avg. RMSE")
plt.xlabel("Degrees")
location = 0
plt.legend()
plt.show()

### To find the avg_rsme on each fold with degree=12 and given array of alphas

In [ ]:
rmse_list = []
avg_rmse_list = []

rmse_list_train = []
avg_rmse_list_train = []
for alpha in alphas:
    rmse_scores = []
    avg_rmse = 0
    
    rmse_scores_train = []
    avg_rmse_train = 0
  
    for train_index, val_index in kf.split(X):
        # Split the data into training and Validation sets
        X_train, X_Val = X[train_index], X[val_index]
        y_train, y_Val = y[train_index], y[val_index]
        
        # Using StandardScaler to Normalize data
        X_train_scaled = scaler.fit_transform(X_train)
        X_Val_scaled = scaler.transform(X_Val)
        
        y_train_scaled = scaler.transform(y_train)
        y_Val_scaled = scaler.transform(y_Val)
        
        # polynomial features
        poly = PolynomialFeatures(degree=12)
        X_train_poly = poly.fit_transform(X_train_scaled)
        X_Val_poly = poly.transform(X_Val_scaled)
        
        # Ridge model
        model = Ridge(alpha=alpha)
        model.fit(X_train_poly, y_train_scaled)

        #  predictions on the Validation test set
        y_pred_Val = model.predict(X_Val_poly)
        y_pred_Val_inv = scaler.inverse_transform(y_pred_Val)
        
        #  predictions on the Train set
        y_pred_train = model.predict(X_train_poly)
        y_pred_train_inv = scaler.inverse_transform(y_pred_train)

        # To find the RMSE score 
        rmse = np.sqrt(mean_squared_error(y_Val, y_pred_Val_inv))
        rmse_scores.append(rmse)
        
        rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train_inv))
        rmse_scores_train.append(rmse_train)
        
        # Append the RMSE score to the average
        avg_rmse += rmse
        avg_rmse_train += rmse_train

    #these are the errors on the Val set of the training data
    avg_rmse /= kf.get_n_splits()
    rmse_list.append(rmse_scores)
    avg_rmse_list.append(avg_rmse)
    
    avg_rmse_train /= kf.get_n_splits()
    rmse_list_train.append(rmse_scores_train)
    avg_rmse_list_train.append(avg_rmse_train)

In [ ]:
#rmse_list

In [ ]:
avg_rmse_list

In [ ]:
plt.figure(figsize=(10, 6))   
plt.title("Graph 2: Avg RMSE vs. Alphas", size=16)

plt.scatter(alphas, avg_rmse_list)
plt.plot(alphas, avg_rmse_list, c="red", label='CV_test')

plt.scatter(alphas, avg_rmse_list_train)
plt.plot(alphas, avg_rmse_list_train, c="green", label='CV_train')

plt.ylabel("Avg. RMSE")
plt.xlabel("Alphas")

plt.xscale('log')

location = 0
plt.legend()
plt.show()

In [ ]:
# print the index of alpha with the minimum avg_rsme.
min(range(len(avg_rmse_list)), key=lambda x : avg_rmse_list[x])

### Consider d*=6 for training the model on train.dat and predit on train.dat & the plotting of Graph 3 and Calculation of the train RMSE

In [ ]:
# Standardize the Input Feature and Target Variable
X_train_scaled = scaler.fit_transform(X)
y_train_scaled = scaler.transform(y)

# Create polynomial features
poly = PolynomialFeatures(degree=6)
X_train_poly = poly.fit_transform(X_train_scaled)

# Fit a linear regression model
model_optD = LinearRegression(fit_intercept=True)
model_optD.fit(X_train_poly, y_train_scaled)

# predictions on the train.dat and test.dat set
y_pred_train = model_optD.predict(X_train_poly)
y_pred_train_inv = scaler.inverse_transform(y_pred_train)

rmse_train = np.sqrt(mean_squared_error(y, y_pred_train_inv))

In [ ]:
rmse_train

In [ ]:
model_optD.coef_

### Calculation of the test RMSE

In [ ]:
#test.dat set
X_test_scaled = scaler.transform(X_test)
y_test_scaled = scaler.transform(y_test)

X_test_poly = poly.transform(X_test_scaled)

y_pred_test = model_optD.predict(X_test_poly)
y_pred_test_inv = scaler.inverse_transform(y_pred_test)

rmse_test = np.sqrt(mean_squared_error(y_test,y_pred_test_inv))

In [ ]:
rmse_test

In [ ]:
#conversions to plot the data 
train_dict = {}

for i in range(df_train['Year'].shape[0]):
    features = df_train['Year'][i]
    output = y_pred_train_inv[i][0]
    train_dict[features] = output
    
train_dict = dict(sorted(train_dict.items()))
#print(train_dict)

In [ ]:
plt.figure(figsize=(10, 10))
plt.title("Graph 3 for best d(d*): Age population vs. Year)", size=16)
plt.scatter(X, y)
plt.plot(np.array(list(train_dict.keys())), np.array(list(train_dict.values())), c="red", label="Predicted Population on training data")
plt.ylabel("Population")
plt.xlabel("Years")
location = 0
plt.legend()
plt.show()

### Consider λ*=e**3 and d=12 for training the model on train.dat and predit on train.dat & the plotting of Graph 4 and Calculation of the train RMSE

In [ ]:
# Standardize the Input Feature and Target Variable
X_train_scaled = scaler.fit_transform(X)
y_train_scaled = scaler.transform(y)

# Create polynomial features
poly = PolynomialFeatures(degree=12)
X_train_poly = poly.fit_transform(X_train_scaled)

# Fit a Ridge model
model_optL = Ridge(alpha=e**-3, fit_intercept=True)
model_optL.fit(X_train_poly, y_train_scaled)

# predictions on the test.dat set
y_pred_train_l = model_optL.predict(X_train_poly)
y_pred_train_inv_l = scaler.inverse_transform(y_pred_train_l)

rmse_train_l = np.sqrt(mean_squared_error(y, y_pred_train_inv_l))

In [ ]:
rmse_train_l

In [ ]:
model_optL.coef_

### Calculation of the test RMSE

In [ ]:
#test.dat set
X_test_scaled = scaler.transform(X_test)
y_test_scaled = scaler.transform(y_test)

X_test_poly = poly.transform(X_test_scaled)

y_pred_test_l = model_optL.predict(X_test_poly)
y_pred_test_inv_l = scaler.inverse_transform(y_pred_test_l)

rmse_test_l = np.sqrt(mean_squared_error(y_test, y_pred_test_inv_l))

In [ ]:
rmse_test_l

In [ ]:
#conversions to plot the data 
train_dict_l = {}

for i in range(df_train['Year'].shape[0]):
    features = df_train['Year'][i]
    output = y_pred_train_inv[i][0]
    train_dict_l[features] = output
    
train_dict_l = dict(sorted(train_dict_l.items()))
#print(train_dict_l)

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Graph 4 for best Lambda: Age population vs. Year)", size=16)
plt.scatter(X, y)

plt.plot(np.array(list(train_dict_l.keys())), np.array(list(train_dict_l.values())), c="red", label="Predicted Population on training data")
plt.ylabel("Population")
plt.xlabel("Years")
location = 0
plt.legend()
plt.show()